In [10]:
import numpy as np
import ephem

def sech(x):
    return np.cosh(np.deg2rad(x))**(-1)

def sin(x):
    return np.sin(np.deg2rad(x))

def airmass(alt):
    X = 1./( sin(alt + 244./(165. + 47.*alt**1.1) ) )
    return X

def fobject(name, ra, dec, epoch = 2000.):
    o = ephem.readdb("{},f|M|F7,{},{},2.02,{}".format(name, ra,dec,epoch))
    return o

def compute_AM(obj, obs, start = '2018/3/21 05:00', end = '2018/3/21 13:00', timezone='US/Central', debug=False):
    moon = ephem.Moon()
    obs.date = start
    t = []
    while obs.date <= ephem.Date(end):
        obs.date += ephem.minute * 5.
        moon.compute(obs)
        obj.compute(obs)
        sep = ephem.separation((moon.az, moon.alt), (obj.az, obj.alt))
        lt = ephem.localtime(obs.date).ctime()
        X = airmass(np.rad2deg( obj.alt ))
        t.append([obs.date.datetime(), np.rad2deg( obj.alt ), np.rad2deg( obj.az ), X, np.rad2deg( sep )])
    return np.array(t)

def compute_eff_AM(obj, obs, start = '2018/3/21 05:00', end = '2018/3/21 13:00'):
    aa = compute_AM(obj, obs, start, end)
    return np.mean( [a[3] for a in aa] )
    
def getSetAndRise(obsin, date, horizon= '-18.'):
    sun = ephem.Sun()
    obs = ephem.Observer()
    obs.lon = obsin.lon
    obs.lat = obsin.lat
    obs.elevation = obsin.elevation
    obs.date = date
    obs.date = date
    obs.pressure = 0
    obs.horizon = horizon
    r1 = obs.next_rising(ephem.Sun())
    s1 = obs.previous_setting(ephem.Sun())
    return  s1.datetime() ,  r1.datetime()
       
# this is Subaru
obs =  ephem.Observer()                                                                                                                                   
obs.name = "Subaru"
obs.lon = str("-155.476667")
obs.lat = str("19.825556")
obs.elevation = 4139


In [11]:
from astropy.io import ascii

targetfield_table = ascii.read(\
"""
name,    ra,      dec,     cost
Umi_A01, 228.200, +67.500, 200
Umi_A02, 228.200, +67.500, 200
Umi_A03, 228.200, +67.500, 200
Umi_B01, 226.300, +67.500, 200
Umi_B02, 226.300, +67.500, 200
Umi_B03, 226.300, +67.500, 200
Umi_C01, 226.000, +66.900, 200
Umi_C02, 226.000, +66.900, 200
Umi_C03, 226.000, +66.900, 200
Umi_D01, 228.100, +66.955, 200
Umi_D02, 228.100, +66.955, 200
Umi_D03, 228.100, +66.955, 200
GE_D00,  150.696720,     2.822468, 100
GE_D01,  150.718438,     2.812009, 100
GE_D02,  150.723801,     2.788509, 100
GE_D03,  150.708772,     2.769663, 100
GE_D04,  150.684668,     2.769663, 100
GE_D05,  150.669639,     2.788509, 100
GE_D06,  150.675002,     2.812009, 100
GE_D07,  150.696720,     2.822468, 100
"""
)
targetfield_table.write("targetfields.txt", format="ascii.fixed_width", overwrite=True)

In [17]:
print( getSetAndRise(obs, "2021/2/10") )
print( getSetAndRise(obs, "2021/2/11") )

(datetime.datetime(2021, 2, 9, 5, 34, 6, 54059), datetime.datetime(2021, 2, 10, 15, 37, 32, 120368))
(datetime.datetime(2021, 2, 10, 5, 34, 31, 15194), datetime.datetime(2021, 2, 11, 15, 37, 7, 923205))


In [18]:
# start of night  5:34 UT
# start of night 15:37 UT
observingslot_table = ascii.read(\
"""
name,           start,             end
20210201_0600, 2021/2/10 06:00:00, 2021/2/10 07:00:00 
20210201_0700, 2021/2/10 07:00:00, 2021/2/10 08:00:00 
20210201_0800, 2021/2/10 08:00:00, 2021/2/10 09:00:00 
20210201_0900, 2021/2/10 09:00:00, 2021/2/10 10:00:00 
20210201_1000, 2021/2/10 10:00:00, 2021/2/10 11:00:00 
20210201_1100, 2021/2/10 11:00:00, 2021/2/10 12:00:00 
20210201_1200, 2021/2/10 12:00:00, 2021/2/10 13:00:00 
20210201_1300, 2021/2/10 13:00:00, 2021/2/10 14:00:00 
20210201_1400, 2021/2/10 14:00:00, 2021/2/10 15:00:00
20210210_0600, 2021/2/10 06:00:00, 2021/2/10 07:00:00
20210210_0700, 2021/2/10 07:00:00, 2021/2/10 08:00:00
20210210_0800, 2021/2/10 08:00:00, 2021/2/10 09:00:00
20210210_0900, 2021/2/10 09:00:00, 2021/2/10 10:00:00
20210210_1000, 2021/2/10 10:00:00, 2021/2/10 11:00:00
20210210_1100, 2021/2/10 11:00:00, 2021/2/10 12:00:00
20210210_1200, 2021/2/10 12:00:00, 2021/2/10 13:00:00
20210210_1300, 2021/2/10 13:00:00, 2021/2/10 14:00:00
20210210_1400, 2021/2/10 14:00:00, 2021/2/10 15:00:00
20210211_0600, 2021/2/11 06:00:00, 2021/2/11 07:00:00 
20210211_0700, 2021/2/11 07:00:00, 2021/2/11 08:00:00 
20210211_0800, 2021/2/11 08:00:00, 2021/2/11 09:00:00 
20210211_0900, 2021/2/11 09:00:00, 2021/2/11 10:00:00 
20210211_1000, 2021/2/11 10:00:00, 2021/2/11 11:00:00 
20210211_1100, 2021/2/11 11:00:00, 2021/2/11 12:00:00 
20210211_1200, 2021/2/11 12:00:00, 2021/2/11 13:00:00 
20210211_1300, 2021/2/11 13:00:00, 2021/2/11 14:00:00 
20210211_1400, 2021/2/11 14:00:00, 2021/2/11 15:00:00
""")
observingslot_table.write("observingslot.txt", format="ascii.fixed_width", overwrite=True)

In [19]:
from astropy.table import Table
visibility_table = Table(names=["targetfield", "observingslot", "airmass"], dtype=["U14", "U14", float])

for t in targetfield_table:
    obj = fobject(t["name"], t["ra"], t["dec"]) 
    for r in observingslot_table:
        # get effective airmass over the observing block
        eff_AM = compute_eff_AM(obj, obs, start=r["start"], end=r["end"])
        if eff_AM < 2:
               visibility_table.add_row([t["name"], r["name"], eff_AM])

/Users/mxhf/.pyenv/versions/3.6.0/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [20]:
visibility_table.write("visibilities.txt", format="ascii.fixed_width", overwrite=True)